# Tick 2

* [NumPy warmup exercises](#notes2) &mdash; not assessed
* [Tick 2a](#tick2a) &mdash; worth 1 mark
* [Tick 2b](#tick2b) &mdash; worth 1 mark
* [Tick 2*](#tick2star) &mdash; not assessed

# NumPy warmup exercises (not assessed)<span id="notes2"></span>

<div class="alert alert-warning">
    These are optional warmup exercises, to get you used to numpy. 
When you submit an answer, you'll also be shown a model answer.
    As with the Python warmup exercises, each question has a code (in brackets) 
    which you should use to fetch to see the model answer.
    </div>


Use the following autograder settings:

```
import ucamcl
GRADER = ucamcl.autograder('https://markmy.solutions', course='scicomp').subsection('notes2')
```

**Exercise (ex1)** from section 2.2.2. Here is some standard Python code:
```
import math, random
x = random.uniform(-1, 1)
y = random.uniform(-1, 1)
d = math.sqrt(x**2 + y**2)
```
We'd like to repeat this a million times, and find the mean and standard deviation of the `d` values. Implement this using numpy vectorized code.

**Exercise (ex2)** from section 2.3. For a numpy matrix `a`, what is the relationship between `a.shape` and `len(a)`?

**Exercise (ex3)** from section 2.3. Look up the numpy help for [`np.arange`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html) and [`reshape`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.reshape.html), and use these functions to produce the $3\times5$ matrix
$$
b = \left( \begin{matrix} 
1 & 2 & 3 & 4 & 5\\
6 & 7 & 8 & 9 & 10\\
11 & 12 & 13 & 14 & 15
\end{matrix} \right)
$$

Look up the help for [`np.sum`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html), and compute the length-5 vector of column sums and the length-3 vector of row sums.

**Exercise (ex4)** from section 2.3. Find two different ways to use numpy to create the column vector
`[[1],[2],...,[n]]`.

**Exercise (ex5)** from section 2.3. A [permutation matrix](https://en.wikipedia.org/wiki/Permutation_matrix) is a square matrix of 0s and 1s,
where each row contains exactly one 1, and each column likewise. (The code snippet in section 2.3 of notes, for 'advanced indexing', creates a $3\times3$ permutation matrix.)

Write code to generate a random $n\times n$ permutation matrix.

**Exercise (ex6)** from section 2.2.2. In a [previous exercise](#lindley) you wrote a Pythonic simulator for a queue, based on the recursion
$$
q_{t+1} = \max(q_t + a_t - C, 0).
$$
It can be proved that another way to get the same answer is with the formula
$$
q_t = q_0 + x_t - \min(0, y_t)
$$
where
$$
x_t = \sum_{u=0}^{t-1} (a_u-C)
\quad\text{and}\quad
y_t = \min_{1 \leq u \leq t} (q_0 + x_u).
$$
Given a vector $a=[a_0,a_1,\dots,a_{t-1}]$,
* compute $x=[x_1,x_2,\dots,x_t]$ using [`np.cumsum`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cumsum.html)
* compute $y=[y_1,y_2,\dots,y_t]$ by [accumulating](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ufunc.accumulate.html) the function `np.minimum`
* compute $q=[q_1,q_2,\dots,q_t]$, and check your answer against your Pythonic code.

**Exercise (ex7)** from section 2.4

When we used numerical optimization in section 2.4, to find the minimum of
$$
f(x) = x -3 x^2 + x^4,
$$
we used the initial guess $x_0=0.5$ and found the local minimum at $x=1.13$.
Now, we'll look for a global minimum. It's good practice to search the parameter space
randomly, to avoid [Moiré effects](https://en.wikipedia.org/wiki/Moir%C3%A9_pattern).
* Create a vector `x` consisting of values randomly chosen in the interval $[-2,2]$
* Create a vector `optx` containing the result of running `scipy.optimize.fmin`starting at each of the `x` values.
* Plot your answers with `plt.plot(x, optx, marker='o')`. 

_You can turn off the diagnostic output with the option
`fmin(disp=False)`. You'll need to sort the points before plotting, else the line will go back and forth across the plot._

# Tick 2a. Econophysics simulator<span id="tick2a"></span>

This section is worth 1 mark. Use these autograder settings:
```
import ucamcl
GRADER = ucamcl.autograder('https://markmy.solutions', course='scicomp').subsection('tick2a')
```

<div class="alert alert-warning">This assignment tests your vectorized thinking. You will be asked to run simulations on a population of hundreds of thousands of individuals, over many timesteps. <strong>YOUR CODE MUST USE NUMPY VECTORIZED OPERATIONS</strong> rather than iterating over the population. You may use Python iteration over timesteps.

## Introduction

Economic inequality is one of the defining social issues of our age. Yet we have a poor grasp of the scale of inequality,
as [described in Scientific American](https://www.scientificamerican.com/article/economic-inequality-it-s-far-worse-than-you-think/) 
and nicely shown in [this video](https://www.youtube.com/watch?v=QPKKQnijnsM):
[<img src="res/inequality.png" style="height:16em">](https://www.youtube.com/watch?v=QPKKQnijnsM)

How does inequality arise? Is it an inevitable outcome of liberal economics, and if so how can it be  mitigated by economic policy? These questions [have been studied by economists](https://link.springer.com/article/10.1140/epjst/e2016-60162-3) and more recently 
[by](https://phys.org/news/2007-04-world-economies-similarities-economic-inequality.html)
[physicists](https://arxiv.org/abs/1606.06051).
In this assignment you will investigate a simple "econophysics" model of inequality.

Here is a simple model. There are $N$ individuals in the population, each with an initial wealth of &pound;1. Every timestep, we randomly group them into $N/2$ pairs. (Assume $N$ is even.) For every pair, we simulate an economic exchange, as follows. Let the two paired individuals have wealth $v$ and $w$, and update their wealth according to
$$
v_{\text{new}} = R(v+w),
\quad
w_{\text{new}} = (1-R)(v+w)
$$
where $R$ is a random number in $[0,1]$, chosen independently for every pair and at every timestep.
This model is loosely inspired by the physics of gases, in which two gas molecules exchange a random amount of energy whenever they collide.

We can measure inequality with the [Gini coefficient](https://en.wikipedia.org/wiki/Gini_coefficient),
$$
G = 2\frac{\sum_{i=1}^N i\, w_{(i)}}{N \sum_i w_{(i)}} - \Big(1 + \frac{1}{N}\Bigr)
$$
where $w_{(1)}$ is the smallest value, $w_{(2)}$ the second smallest etc. If everyone has the same wealth then $G=0$; if one person has all the wealth then $G=1-1/N$.

## Questions

**Question 1.** The model needs us to randomly group the population into $N/2$ pairs. 
We can do this by randomly permuting the vector $[0,...,N-1]$, letting the vector `m1` consist of the first $N/2$ integers and `m2` consist of the rest, and interpreting it as "`m1[i]` is paired with `m2[i]`".

Write a function `pairs(N)` that returns a tuple `(m1,m2)` where `m1` and `m2` are both vectors of length $N/2$ as described above. For example, if you run `pairs(6)`, you might get the output
```
(array[(3, 0, 1]), array([2, 4, 5]))
```
To submit your answer,
```
q = GRADER.fetch_question('q1')
m1,m2 = pairs(q.n)
ans = {'n': len(np.unique(np.concatenate([m1,m2]))), 's': np.std(np.abs(m1-m2))}
GRADER.submit_answer(q, ans)
```

**Question 2.** Write a function `kinetic_exchange(v,w)` which takes two wealth vectors `v` and `w`, each of length $N/2$, and returns a tuple `(vnew, wnew)` with two new vectors, according to the kinetic exchange model. To submit your answer,
```
q = GRADER.fetch_question('q2')
v,w = np.linspace(1,5,q.n), np.linspace(1,2,q.n)**q.p
vnew,wnew = kinetic_exchange(v,w)
ans = {'m1': np.mean(vnew), 's2': np.std(wnew)}
GRADER.submit_answer(q, ans)
```

**Question 3.** Write a function `gini(w)` which takes a vector `w` and returns the Gini coefficient. To submit your answer,
```
q = GRADER.fetch_question('q3')
w = np.linspace(0,1,q.n)**q.p
g = gini(w)
GRADER.submit_answer(q, {'g': g})
```

**Question 4.** Write a function `sim(N, T)` which runs the kinetic exchange model on a population of $N$ individuals for $T$ timesteps. It should return a pair `(w, gs)` where `w` is the wealth vector after $T$ timesteps, and `gs` is a length $T$ vector where `gs[i]` is the Gini coefficient at timestep $i$. To submit your answer,
```
q = GRADER.fetch_question('q4')
w,gs = sim(q.n, q.t)
ans = {'gm': np.mean(gs[int(q.t/2):]), 'gs': np.std(gs[int(q.t/2):]), 'ws': np.std(w)}
GRADER.submit_answer(q, ans)
```

**Question 5.** Simulate a population of 500,000 over 30 iterations. Plot the Gini coefficient as a function of timestep.
To be precise, if $w_t$ is the wealth vector after $t$ timesteps then you should plot $\textsf{gini}(w_t)$ on the $y$-axis and $t$ on the $x$-axis.
You don't have to submit your plot, but it may be assessed in the ticking session. _Your plot should look something like this:_

![gini coefficient](res/ass2_q5.png)

# Tick 2b. Economic mobility<span id="tick2b"></span>

This section is worth 1 mark. Use these autograder settings:
```
import ucamcl
GRADER = ucamcl.autograder('https://markmy.solutions', course='scicomp').subsection('tick2b')
```

<div class="alert alert-warning">This assignment tests your vectorized thinking. You will be asked to run simulations on a population of hundreds of thousands of individuals, over many timesteps. <strong>YOUR CODE MUST USE NUMPY VECTORIZED OPERATIONS</strong> rather than iterating over the population. You may use Python iteration over timesteps.

## Introduction

Some degree of inequality might be acceptable if economic mobility were high, i.e. if everyone had similar chances of reaching either end of the wealth distribution. Economic mobility is often measured by splitting the population into five equal brackets, and measuring the chance of moving between brackets. From the [Wikipedia article on economic mobility](https://en.wikipedia.org/wiki/Economic_mobility):

> in terms of relative mobility [a report](https://www.brookings.edu/research/economic-mobility-of-families-across-generations/) stated: "contrary to American beliefs about
> equality of opportunity, a child’s economic position is heavily influenced by that of his
> or her parents." 42% of children born to parents in the bottom fifth of the income
> distribution ("quintile") remain in the bottom, while 39% born to parents in the top fifth
> remain at the top.

Let's measure economic mobility by recording the wealth distribution at one timepoint, and again some number of timesteps later, splitting the two distributions into quintiles, and counting what fraction of the population moved by more than one quintile from beginning to end. (In each timestep a median individual might find their wealth increasing or decreasing by around 50%, so one timestep corresponds roughly to several years of human life.) For example, if we have a population of 5000 and we draw up a matrix $A$ where $A_{i j}$ is the number of people who start in quintile $i$ and end up in quintile $j$, we might get
$$
A = \left( \begin{matrix}
344& 313& 243& 100&   0\\
266& 261& 302& 167&   4\\
212& 260& 225& 272&  31\\
147& 143& 183& 331& 196\\
 31&  23&  47& 130& 769
\end{matrix} \right)
$$
(A quick check: the row sums and column sums are all 1000.) The number who moved by more than one quintile is 1148, which is 23% of the population.

## Questions

**Question 6.** In a perfectly mobile economy, where everyone has equal chance of reaching any quintile, what fraction of people are expected to move by more than one quintile?
```
q = GRADER.fetch_question('q6')
GRADER.submit_answer(q, your_answer)
```

**Question 7.** Write a function `mobility(v,w)` that returns the proportion of people who moved by more than one quintile, where `v[i]` and `w[i]` measure respectively the wealth of individual $i$ at the beginning and end of a time period. _Hint: look up [`np.percentile`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.percentile.html#numpy.percentile) and [`np.digitize`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.digitize.html)._
```
# Submitting your answer:
q = GRADER.fetch_question('q7')
v,w = np.arange(q.n)**q.a, np.arange(q.n)**q.a * np.random.random(q.n)
GRADER.submit_answer(q, mobility(v,w))
```

**Question 8.** Simulate the kinetic exchange model from Assignment 2a long enough for it to stabilize; call this time $T$, and let the wealth vector be $w_T$. Run the model $t$ timesteps further to time $T+t$, find the wealth vector $w_{T+t}$, and compute $\textsf{mobility}(w_T, w_{T+t})$. It's up to you to decide how to judge stabilization; you don't have to explain your method but you do have to submit a correct answer.
```
# Submitting your answer:
q = GRADER.fetch_question('q8')
# For a population size q.n, measure mobility over from time T to time T+q.t
GRADER.submit_answer(q, your_answer)
```

**Question 9.** Plot the social mobility for a population of 500,000 as in Question 8, over a sequence of timesteps. To be precise, plot $\textsf{mobility}(w_T, w_{T+t})$ on the $y$-axis and $t$ on the $x$-axis.
You don't have to submit your plot, but it may be assessed in the ticking session. _Your plot should look something like this:_

![mobility](res/ass2_q9.png)

<img src="res/mobility.svg" style="height:18em"/>

# Tick 2* (not asssessed)<span id="tick2star"></span>

<div class="alert alert-warning">
    These are optional further investigations into tax policy, using out econophysics simulator.
    </div>
    
Use the following autograder settings:
```
import ucamcl
GRADER = ucamcl.autograder('https://markmy.solutions', course='scicomp').subsection('tick2star')
```

We will investigate variations on the kinetic exchange model, and alternative metrics. The first step is to rewrite the simulator to be more modular. Rewrite the simulator as a function
```
def sim(w0, T, update, metrics): 
    ...
    return (w, res)
```
where
* `w0` is either an integer or an initial wealth vector; if `w0` is integer then use initial wealth vector `np.ones(w0)`
* `T` is the number of timesteps to simulate
* `update` is a function to update wealth, with the same signature as `kinetic_exchange`
* `metrics` is a list of functions, each with the same signature as `gini`
* `w` is the wealth vector after `T` timesteps
* `res` is a `T x len(metrics)` matrix, recording the value of each of the metrics at each timestep

Thus the answer to Tick 2a question 5 would be computed by
```
sim(500000, 30, kinetic_exchange, [gini])
```
and the answer to Tick 2b question 9 would be computed by
```
T = ...
w0,_ = sim(500000, T, kinetic_exchange, [])
_,r = sim(w0, 30, kinetic_exchange, [lambda w: mobility(w0,w)])
```

**Exercise (ex1).**
In the kinetic exchange model, the poorest and the richest might swap places after just one transaction, which isn't very likely. Consider a different model for exchange.
As before, suppose that two individuals with wealth $v$ and $w$ respectively are paired, but now let their wealth be updated by
$$
v_{\text{new}} = v + R \min(v,w),
\quad
w_{\text{new}} = w - R \min(v,w)
$$
where $R$ is now a random number in $[-1,1]$, chosen independently for every pair at every timestep. The idea is that each party to the exchange puts up a certain amount of money, but no more than they can afford. Implement an update function to model this.
```
# Submitting your answer:
q = GRADER.fetch_question('ex1')
# your_ans = Gini coefficient after q.t timesteps for a popn of size q.n
GRADER.submit_answer(q, your_ans)
```

**Exercise (ex2).** The Gini coefficient is unfamiliar to many people, and it's easier to communicate "The richest 1% of the population own $x$% of the wealth." Implement a function
```
topk(w, p)
```
which computes what fraction of the total wealth is owned by the top $p$ of the population.
```
# Submitting your answer
q = GRADER.fetch_question('ex2')
# your_ans = fraction of wealth owned by top 1%, after q.t timesteps for a popn of size q.n
GRADER.submit_answer(q, your_ans)
```

**Exercise (ex3).** The government can intervene to reduce inequality. Suppose it levies a tax of say 40% on every exchange, collects all the tax revenue every timestep, and distributes it evenly to the entire population. Here's a concrete example, for a population of size 6.

> 1. Initial wealth values are $[0,2,5,3,1,2]$
> 1. We pair individuals randomly: $(0,2), (5,3), (1,2)$
> 1. Random exchange amounts pre-tax are $(0,0), (2.6,-2.6), (-0.4,0.4)$
> 1. Exchange amounts post-tax are $(0,0), (2.08,-2.6), (-0.4,0.32)$
> 1. Government revenue is $(2.6-2.08) + (0.4-0.32) = 0.6$
> 1. Government redistributes $0.6/6=0.1$ to each person
> 1. Change in wealth is $[0.1,0.1, 2.18,-2.5, -0.3,0.42]$
> 1. New wealth vector is $[0.1, 2.1, 7.18, 0.5, 0.7, 2.42]$

Implement an update function to model this.
```
# Submitting your answer
q = GRADER.fetch_question('ex3')
# your_ans = fraction of wealth owned by top 1%, at tax rate q.taxrate,
# after q.t timesteps for a popn of size q.n
GRADER.submit_answer(q, your_ans)
```

**Further investigations.** 

* Is there a tradeoff between inequality and social mobility? Try different tax rates, measure the inequality and the social mobility, and plot your results.

* The economist [Thomas Piketty argues](https://en.wikipedia.org/wiki/Capital_in_the_Twenty-First_Century) that we have entered an age where the return on capital is greater than the growth due to income, and that this leads to higher inequality. We could incorporate income into the model by assigning each individual $i$ a per-timestep income $g_i$, where the $g_i$ are randomly chosen _a priori_. We could incorporate return on capital into the model, by multiplying wealth by a growth factor every timestep (and rescaling income to account for inflation). Investigate what happens when we combine these two extensions. How well correlated are income and wealth? How does the relationship depend on capital growth rate? Do you agree with Piketty? Does taxation alter the relationship?